In [1]:


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.experiments.experiment_analysis_results import *

In [2]:
def print_step_num_multi_compare_with_group(results_compares, results, step_num: int = 800, reverse: bool = False):
    wdl = [{} for _ in results_compares]
    source = {}
    source_compares = [{} for _ in results_compares]
    tar_type = ""
    for key in results.keys():
        config = results[key]["config"]
        if len(config) == 11:
            (_, _, _, _, tar_type, tar_domain, tar_dim, tar_idx,
             sample_num, _, sample_idx) = config
        else:
            (tar_type, tar_domain, tar_dim, tar_idx, sample_idx) = config
        top_best_y = [np.max(item["step_history"][:step_num]) for item in results[key]["metric"]]
        top_best_y_compares = [[np.max(item["step_history"][:step_num]) for item in results_compare[key]["metric"]] for
                               results_compare in results_compares]
        if tar_domain not in source.keys():
            source[tar_domain] = {}
        if tar_dim not in source[tar_domain].keys():
            source[tar_domain][tar_dim] = {}
        if tar_idx not in source[tar_domain][tar_dim].keys():
            source[tar_domain][tar_dim][tar_idx] = []
        source[tar_domain][tar_dim][tar_idx] += top_best_y
        for source_idx in range(len(source_compares)):
            if tar_domain not in source_compares[source_idx].keys():
                source_compares[source_idx][tar_domain] = {}
            if tar_dim not in source_compares[source_idx][tar_domain].keys():
                source_compares[source_idx][tar_domain][tar_dim] = {}
            if tar_idx not in source_compares[source_idx][tar_domain][tar_dim].keys():
                source_compares[source_idx][tar_domain][tar_dim][tar_idx] = []
            source_compares[source_idx][tar_domain][tar_dim][tar_idx] += top_best_y_compares[source_idx]
    tar_idx_num = problem_settings["training_gate_ins_num"] if tar_type == "gate_train" else problem_settings[
        "valid_ins_num"]
    tar_domains = train_problem_domains if tar_type == "gate_train" else valid_problem_domains
    large_mean = [0 for _ in results_compares]
    all_means = [[] for _ in range(len(source_compares) + 1)]
    for tar_domain in tar_domains.keys():
        for tar_dim in problem_settings["valid_dims"]:
            for tar_idx in range(tar_idx_num):
                print(f"{tar_domain}\t{tar_dim}\t{tar_idx}", end="\t")
                print(
                    "{:.5g}±{:.3g}".format(
                        np.mean(source[tar_domain][tar_dim][tar_idx]),
                        np.std(source[tar_domain][tar_dim][tar_idx])
                    ),
                    end="\t"
                )
                for source_idx in range(len(source_compares)):
                    if tar_domain not in wdl[source_idx].keys():
                        wdl[source_idx][tar_domain] = [0, 0, 0]
                    if ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                source[tar_domain][tar_dim][tar_idx],
                                alternative="greater")[1] < 0.05:
                        compare = "↑"
                        wdl[source_idx][tar_domain][0] += 1
                    elif ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                  source[tar_domain][tar_dim][tar_idx],
                                  alternative="less")[1] < 0.05:
                        compare = "↓"
                        wdl[source_idx][tar_domain][2] += 1
                    else:
                        compare = "→"
                        wdl[source_idx][tar_domain][1] += 1
                    print(
                        "{:.5g}±{:.3g} {}".format(
                            np.mean(source_compares[source_idx][tar_domain][tar_dim][tar_idx]),
                            np.std(source_compares[source_idx][tar_domain][tar_dim][tar_idx]),
                            compare
                        ),
                        end="\t"
                    )
                    if np.mean(source[tar_domain][tar_dim][tar_idx]) < np.mean(
                            source_compares[source_idx][tar_domain][tar_dim][tar_idx]):
                        large_mean[source_idx] += 1
                    all_means[source_idx].append(np.mean(source_compares[source_idx][tar_domain][tar_dim][tar_idx]))
                all_means[-1].append(np.mean(source[tar_domain][tar_dim][tar_idx]))
                print()
        print(f"{tar_domain}\tW-D-L\t\t", end="\t")
        for source_idx in range(len(source_compares)):
            if reverse:
                print(
                    f"{wdl[source_idx][tar_domain][2]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][0]}",
                    end="\t")
            else:
                print(
                    f"{wdl[source_idx][tar_domain][0]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][2]}",
                    end="\t")
        print()
    for tar_domain in tar_domains.keys():
        print(f"{tar_domain}", end="\t")
        for source_idx in range(len(source_compares)):
            if reverse:
                print(
                    f"{wdl[source_idx][tar_domain][2]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][0]}",
                    end="\t")
            else:
                print(
                    f"{wdl[source_idx][tar_domain][0]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][2]}",
                    end="\t")
        print()

In [3]:
def analysis_step_num_multi_compare(results_compares, results, step_num: int = 800, reverse: bool = False,
                                    show_goal_diff=False):
    wdl = [[0, 0, 0] for _ in results_compares]
    source = {}
    source_compares = [{} for _ in results_compares]
    tar_type = ""
    for key in results.keys():
        config = results[key]["config"]
        if len(config) == 11:
            (_, _, _, _, tar_type, tar_domain, tar_dim, tar_idx,
             sample_num, _, sample_idx) = config
        else:
            (tar_type, tar_domain, tar_dim, tar_idx, sample_idx) = config
        top_best_y = [np.max(item["step_history"][:step_num]) for item in results[key]["metric"]]
        top_best_y_compares = [[np.max(item["step_history"][:step_num]) for item in results_compare[key]["metric"]] for
                               results_compare in results_compares]
        if tar_domain not in source.keys():
            source[tar_domain] = {}
        if tar_dim not in source[tar_domain].keys():
            source[tar_domain][tar_dim] = {}
        if tar_idx not in source[tar_domain][tar_dim].keys():
            source[tar_domain][tar_dim][tar_idx] = []
        source[tar_domain][tar_dim][tar_idx] += top_best_y
        for source_idx in range(len(source_compares)):
            if tar_domain not in source_compares[source_idx].keys():
                source_compares[source_idx][tar_domain] = {}
            if tar_dim not in source_compares[source_idx][tar_domain].keys():
                source_compares[source_idx][tar_domain][tar_dim] = {}
            if tar_idx not in source_compares[source_idx][tar_domain][tar_dim].keys():
                source_compares[source_idx][tar_domain][tar_dim][tar_idx] = []
            source_compares[source_idx][tar_domain][tar_dim][tar_idx] += top_best_y_compares[source_idx]
    tar_idx_num = problem_settings["training_gate_ins_num"] if tar_type == "gate_train" else problem_settings[
        "valid_ins_num"]
    tar_domains = train_problem_domains if tar_type == "gate_train" else valid_problem_domains
    large_mean = [0 for _ in results_compares]
    all_means = [[] for _ in range(len(source_compares) + 1)]
    for tar_domain in tar_domains.keys():
        for tar_dim in problem_settings["valid_dims"]:
            for tar_idx in range(tar_idx_num):
                for source_idx in range(len(source_compares)):
                    if ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                source[tar_domain][tar_dim][tar_idx],
                                alternative="greater")[1] < 0.05:
                        compare = "↑"
                        wdl[source_idx][0] += 1
                    elif ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                  source[tar_domain][tar_dim][tar_idx],
                                  alternative="less")[1] < 0.05:
                        compare = "↓"
                        wdl[source_idx][2] += 1
                    else:
                        compare = "→"
                        wdl[source_idx][1] += 1

                    if np.mean(source[tar_domain][tar_dim][tar_idx]) < np.mean(
                            source_compares[source_idx][tar_domain][tar_dim][tar_idx]) \
                            or (np.mean(source[tar_domain][tar_dim][tar_idx]) == np.mean(
                        source_compares[source_idx][tar_domain][tar_dim][tar_idx]) and reverse):
                        large_mean[source_idx] += 1
                    all_means[source_idx].append(np.mean(source_compares[source_idx][tar_domain][tar_dim][tar_idx]))
                all_means[-1].append(np.mean(source[tar_domain][tar_dim][tar_idx]))
    print(step_num, end="\t")
    for source_idx in range(len(source_compares)):
        if reverse:
            print(f"{wdl[source_idx][2]}-{wdl[source_idx][1]}-{wdl[source_idx][0]}", end="\t")
            if show_goal_diff:
                print(f"{wdl[source_idx][2] - wdl[source_idx][0]}", end="\t")
            print(f"{72 - large_mean[source_idx]}", end="\t")
        else:
            print(f"{wdl[source_idx][0]}-{wdl[source_idx][1]}-{wdl[source_idx][2]}", end="\t")
            if show_goal_diff:
                print(f"{wdl[source_idx][0] - wdl[source_idx][2]}", end="\t")
            print(f"{large_mean[source_idx]}", end="\t")
    print()
    return wdl, large_mean


def analysis_step_num_multi_compare_with_group(results_compares, results, step_num: int = 800, reverse: bool = False):
    wdl = [{} for _ in results_compares]
    source = {}
    source_compares = [{} for _ in results_compares]
    tar_type = ""
    for key in results.keys():
        config = results[key]["config"]
        if len(config) == 11:
            (_, _, _, _, tar_type, tar_domain, tar_dim, tar_idx,
             sample_num, _, sample_idx) = config
        else:
            (tar_type, tar_domain, tar_dim, tar_idx, sample_idx) = config
        top_best_y = [np.max(item["step_history"][:step_num]) for item in results[key]["metric"]]
        top_best_y_compares = [[np.max(item["step_history"][:step_num]) for item in results_compare[key]["metric"]] for
                               results_compare in results_compares]
        if tar_domain not in source.keys():
            source[tar_domain] = {}
        if tar_dim not in source[tar_domain].keys():
            source[tar_domain][tar_dim] = {}
        if tar_idx not in source[tar_domain][tar_dim].keys():
            source[tar_domain][tar_dim][tar_idx] = []
        source[tar_domain][tar_dim][tar_idx] += top_best_y
        for source_idx in range(len(source_compares)):
            if tar_domain not in source_compares[source_idx].keys():
                source_compares[source_idx][tar_domain] = {}
            if tar_dim not in source_compares[source_idx][tar_domain].keys():
                source_compares[source_idx][tar_domain][tar_dim] = {}
            if tar_idx not in source_compares[source_idx][tar_domain][tar_dim].keys():
                source_compares[source_idx][tar_domain][tar_dim][tar_idx] = []
            source_compares[source_idx][tar_domain][tar_dim][tar_idx] += top_best_y_compares[source_idx]
    tar_idx_num = problem_settings["training_gate_ins_num"] if tar_type == "gate_train" else problem_settings[
        "valid_ins_num"]
    tar_domains = train_problem_domains if tar_type == "gate_train" else valid_problem_domains
    large_mean = [{} for _ in results_compares]
    all_means = [[] for _ in range(len(source_compares) + 1)]
    for tar_domain in tar_domains.keys():
        for tar_dim in problem_settings["valid_dims"]:
            for tar_idx in range(tar_idx_num):
                for source_idx in range(len(source_compares)):
                    if tar_domain not in wdl[source_idx].keys():
                        wdl[source_idx][tar_domain] = [0, 0, 0]
                        large_mean[source_idx][tar_domain] = 0
                    if ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                source[tar_domain][tar_dim][tar_idx],
                                alternative="greater")[1] < 0.05:
                        compare = "↑"
                        wdl[source_idx][tar_domain][0] += 1
                    elif ranksums(source_compares[source_idx][tar_domain][tar_dim][tar_idx],
                                  source[tar_domain][tar_dim][tar_idx],
                                  alternative="less")[1] < 0.05:
                        compare = "↓"
                        wdl[source_idx][tar_domain][2] += 1
                    else:
                        compare = "→"
                        wdl[source_idx][tar_domain][1] += 1

                    if np.mean(source[tar_domain][tar_dim][tar_idx]) < np.mean(
                            source_compares[source_idx][tar_domain][tar_dim][tar_idx]) or (
                            np.mean(source[tar_domain][tar_dim][tar_idx]) == np.mean(
                        source_compares[source_idx][tar_domain][tar_dim][tar_idx]) and reverse):
                        large_mean[source_idx][tar_domain] += 1
                    all_means[source_idx].append(np.mean(source_compares[source_idx][tar_domain][tar_dim][tar_idx]))
                all_means[-1].append(np.mean(source[tar_domain][tar_dim][tar_idx]))
    print(np.mean(all_means[-1]))
    for tar_domain in tar_domains.keys():
        print(f"{tar_domain}", end="\t")
        for source_idx in range(len(source_compares)):
            if reverse:
                print(
                    f"{wdl[source_idx][tar_domain][2]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][0]}\t{12 - large_mean[source_idx][tar_domain]}",
                    end="\t")
            else:
                print(
                    f"{wdl[source_idx][tar_domain][0]}-{wdl[source_idx][tar_domain][1]}-{wdl[source_idx][tar_domain][2]}\t{large_mean[source_idx][tar_domain]}",
                    end="\t")
        print()
    return wdl, large_mean

def analysis_step_num_multi_compare_with_dim(results_compares, results, step_num: int = 800, reverse: bool = False):
    wdl = [{} for _ in results_compares]
    source = {}
    source_compares = [{} for _ in results_compares]
    tar_type = ""
    for key in results.keys():
        config = results[key]["config"]
        if len(config) == 11:
            (_, _, _, _, tar_type, tar_domain, tar_dim, tar_idx,
             sample_num, _, sample_idx) = config
        else:
            (tar_type, tar_domain, tar_dim, tar_idx, sample_idx) = config
        top_best_y = [np.max(item["step_history"][:step_num]) for item in results[key]["metric"]]
        top_best_y_compares = [[np.max(item["step_history"][:step_num]) for item in results_compare[key]["metric"]] for
                               results_compare in results_compares]
        if tar_dim not in source.keys():
            source[tar_dim] = {}
        if tar_domain not in source[tar_dim].keys():
            source[tar_dim][tar_domain] = {}
        if tar_idx not in source[tar_dim][tar_domain].keys():
            source[tar_dim][tar_domain][tar_idx] = []
        source[tar_dim][tar_domain][tar_idx] += top_best_y
        for source_idx in range(len(source_compares)):
            if tar_dim not in source_compares[source_idx].keys():
                source_compares[source_idx][tar_dim] = {}
            if tar_domain not in source_compares[source_idx][tar_dim].keys():
                source_compares[source_idx][tar_dim][tar_domain] = {}
            if tar_idx not in source_compares[source_idx][tar_dim][tar_domain].keys():
                source_compares[source_idx][tar_dim][tar_domain][tar_idx] = []
            source_compares[source_idx][tar_dim][tar_domain][tar_idx] += top_best_y_compares[source_idx]
    tar_idx_num = problem_settings["training_gate_ins_num"] if tar_type == "gate_train" else problem_settings[
        "valid_ins_num"]
    tar_domains = train_problem_domains if tar_type == "gate_train" else valid_problem_domains
    large_mean = [{} for _ in results_compares]
    all_means = [[] for _ in range(len(source_compares) + 1)]
    for tar_dim in problem_settings["valid_dims"]:
        for tar_domain in tar_domains.keys():
            for tar_idx in range(tar_idx_num):
                for source_idx in range(len(source_compares)):
                    if tar_dim not in wdl[source_idx].keys():
                        wdl[source_idx][tar_dim] = [0, 0, 0]
                        large_mean[source_idx][tar_dim] = 0

                    if ranksums(source_compares[source_idx][tar_dim][tar_domain][tar_idx],
                                source[tar_dim][tar_domain][tar_idx],
                                alternative="greater")[1] < 0.05:
                        compare = "↑"
                        wdl[source_idx][tar_dim][0] += 1
                    elif ranksums(source_compares[source_idx][tar_dim][tar_domain][tar_idx],
                                  source[tar_dim][tar_domain][tar_idx],
                                  alternative="less")[1] < 0.05:
                        compare = "↓"
                        wdl[source_idx][tar_dim][2] += 1
                    else:
                        compare = "→"
                        wdl[source_idx][tar_dim][1] += 1

                    if np.mean(source[tar_dim][tar_domain][tar_idx]) < np.mean(
                            source_compares[source_idx][tar_dim][tar_domain][tar_idx]) or (
                            np.mean(source[tar_dim][tar_domain][tar_idx]) == np.mean(
                        source_compares[source_idx][tar_dim][tar_domain][tar_idx]) and reverse):
                        large_mean[source_idx][tar_dim] += 1
                    all_means[source_idx].append(np.mean(source_compares[source_idx][tar_dim][tar_domain][tar_idx]))
                all_means[-1].append(np.mean(source[tar_dim][tar_domain][tar_idx]))
    print(np.mean(all_means[-1]))
    for tar_dim in problem_settings["valid_dims"]:
        print(f"{tar_dim}", end="\t")
        for source_idx in range(len(source_compares)):
            if reverse:
                print(
                    f"{wdl[source_idx][tar_dim][2]}-{wdl[source_idx][tar_dim][1]}-{wdl[source_idx][tar_dim][0]}\t{18 - large_mean[source_idx][tar_dim]}",
                    end="\t")
            else:
                print(
                    f"{wdl[source_idx][tar_dim][0]}-{wdl[source_idx][tar_dim][1]}-{wdl[source_idx][tar_dim][2]}\t{large_mean[source_idx][tar_dim]}",
                    end="\t")
        print()
    return wdl, large_mean

In [4]:
def static_wdl(ea="elite_ga", normalize=False):
    num = 132
    k = 4
    b = pickle.loads(open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-random.pkl", "rb").read())
    d = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-OBL-112.pkl", "rb").read())
    e = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-QI-112.pkl", "rb").read())
    f = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-KAES-132.pkl", "rb").read())

    c = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-SVMSS-{num}.pkl", "rb").read())

    a = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea-interpolation-12-{k}-{k}.pkl",
             "rb").read())
    a1 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea-interpolation-random_select-12-{k}-{k}.pkl",
             "rb").read())
    a2 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea-12-{k}-{k}.pkl",
             "rb").read())
    a3 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea_diversity-interpolation-12-{k}-{k}.pkl",
             "rb").read())
    a4 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea_max-interpolation-12-{k}-{k}.pkl",
             "rb").read())
    a5 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea_max_diversity-interpolation-12-{k}-{k}.pkl",
             "rb").read())
    a6 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea_diversity-12-{k}-{k}.pkl",
             "rb").read())
    a7 = pickle.loads(
        open(f"/home/metaron/EMPI_exp/out/eval_result/test-{ea}-no_ea_max_diversity-12-{k}-{k}.pkl",
             "rb").read())
    if normalize:
        b, c, d, e, f = normalize_results(b), normalize_results(c), normalize_results(d), normalize_results(
            e), normalize_results(f)
        a, a1, a2, a3, a4, a5, a6, a7 = normalize_results(a), normalize_results(a1), normalize_results(
            a2), normalize_results(a3), normalize_results(a4), normalize_results(a5), normalize_results(
            a6), normalize_results(a7)
    return a, b, c, d, e, f, a1, a2, a3, a4, a5, a6, a7

In [5]:
def shown_result(a, b, c, d, e, f, a1, a2, a3, a4, a5, a6, a7, final_num=800):
    analysis_step_num_multi_compare_with_group([a4, c, d, f], b, step_num=final_num)
    analysis_step_num_multi_compare([a4, c, d, f], b, step_num=final_num)
    print()
    analysis_step_num_multi_compare_with_group([b, c, d, f], a4, step_num=final_num, reverse=True)
    analysis_step_num_multi_compare([b, c, d, f], a4, step_num=final_num, reverse=True)
    print()
    analysis_step_num_multi_compare_with_group([a1, a6, e], a4, step_num=final_num, reverse=True)
    analysis_step_num_multi_compare([a1, a6, e], a4, step_num=final_num, reverse=True)
    print()
    print()
    final_nums = [i * 100 for i in range(2, 17)]
    num = 132
    print("Compare with Random")
    print(f"Initial Steps={num}")
    analysis_step_num_multi_compare([a4, c, d, f], b, step_num=num, show_goal_diff=True)
    for final_num in final_nums:
        analysis_step_num_multi_compare([a4, c, d, f], b, step_num=final_num, show_goal_diff=True)
    print()
    print("Compare with MPI")
    print(f"Initial Steps={num}")
    analysis_step_num_multi_compare([b, c, d, f], a4, step_num=num, reverse=True, show_goal_diff=True)
    for final_num in final_nums:
        analysis_step_num_multi_compare([b, c, d, f], a4, step_num=final_num, reverse=True, show_goal_diff=True)
    print()
    print("Compare MPI with No Obj and Interpolation")
    print(f"Initial Steps={num}")
    analysis_step_num_multi_compare([a1, a7, e], a4, step_num=num, reverse=True, show_goal_diff=True)
    for final_num in final_nums:
        analysis_step_num_multi_compare([a1, a7, e], a4, step_num=final_num, reverse=True, show_goal_diff=True)
    print()
    print("Compare with MEAN/MEAN_Diversity/MAX_Diversity")
    print(f"Initial Steps={num}")
    analysis_step_num_multi_compare([a, a3, a5], a4, step_num=num, reverse=True, show_goal_diff=True)
    for final_num in final_nums:
        analysis_step_num_multi_compare([a, a3, a5], a4, step_num=final_num, reverse=True, show_goal_diff=True)
    print()

In [6]:
a, b, c, d, e, f, a1, a2, a3, a4, a5, a6, a7 = static_wdl(ea="elite_ga")
a_b, b_b, c_b, d_b, e_b, f_b, a1_b, a2_b, a3_b, a4_b, a5_b, a6_b, a7_b = static_wdl(ea="brkga")

In [ ]:
print_step_num_multi_compare_with_group([b, c, d, f], a4)

In [ ]:
print_step_num_multi_compare_with_group([b_b, c_b, d_b, f_b], a4_b)

In [ ]:
a, b, c, d, e, f, a1, a2, a3, a4, a5, a6, a7 = static_wdl(ea="elite_ga",normalize=True)
a_b, b_b, c_b, d_b, e_b, f_b, a1_b, a2_b, a3_b, a4_b, a5_b, a6_b, a7_b = static_wdl(ea="brkga",normalize=True)

In [ ]:
# Elite_GA
shown_result(a, b, c, d, e, f, a1, a2, a3, a4, a5, a6, a7, final_num=800)


In [ ]:
#BRKGA
shown_result(a_b, b_b, c_b, d_b, e_b, f_b, a1_b, a2_b, a3_b, a4_b, a5_b, a6_b, a7_b)

In [ ]:
shown_result(a_b, b_b, c_b, d_b, e_b, f_b, a1_b, a2_b, a3_b, a4_b, a5_b, a6_b, a7_b)